In [1]:
import pandas as pd
import torch
import os
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm
import torchvision.models as models
from torchvision.models.resnet import ResNet50_Weights
from torchvision.models.densenet import DenseNet161_Weights
from torchvision.models.resnet import ResNeXt50_32X4D_Weights
from torchvision.models.resnet import ResNet101_Weights
from torchvision.models.inception import Inception_V3_Weights

from focal_loss.focal_loss import FocalLoss


In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TEST_IMG_DIR = "ism_project_2023/test"  
MODEL_WEIGHTS_PATH_RESNET = "C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/phase2/best_resnet50_model.pth"
MODEL_WEIGHTS_PATH_DENSENET = "C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/phase2/best_densenet161_model.pth"
MODEL_WEIGHTS_PATH_RESNEXT = "C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/phase2/best_resnext50_model_0.6.pth"
CSV_FILE = "ism_project_2023/test.csv"
NUM_CLASSES=4


In [9]:
model1 = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)  
model1.fc = torch.nn.Linear(in_features=2048, out_features=NUM_CLASSES)
model1.load_state_dict(torch.load(MODEL_WEIGHTS_PATH_RESNET,map_location=torch.device('cpu')))
model1.to(DEVICE)
model1.eval()
print("MODEL LOADED")


MODEL LOADED


In [5]:
model2 = models.resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)  
model2.fc = torch.nn.Linear(in_features=2048, out_features=NUM_CLASSES)
model2.load_state_dict(torch.load(MODEL_WEIGHTS_PATH_RESNEXT,map_location=torch.device('cpu')))
model2.to(DEVICE)
model2.eval()
print("MODEL 2 LOADED")

MODEL 2 LOADED


In [6]:
model3 = models.densenet161(weights=DenseNet161_Weights.IMAGENET1K_V1)  
model3.fc = torch.nn.Linear(in_features=2048, out_features=NUM_CLASSES)
model3.load_state_dict(torch.load(MODEL_WEIGHTS_PATH_DENSENET,map_location=torch.device('cpu')))
model3.to(DEVICE)
model3.eval()
print("MODEL 3 LOADED")

MODEL 3 LOADED


In [7]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
    
])


In [8]:
TEST={"count":[],"probabilities_1":[],"probabilities_2":[],"probabilities_3":[]}

In [16]:
def predict1(image_path, model, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0) 
    image = image.to(DEVICE)
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        TEST["probabilities_1"].append(probabilities)
        predicted_class = probabilities.argmax().item()
    return int(predicted_class)

In [17]:
def predict2(image_path, model, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0) 
    image = image.to(DEVICE)
    with torch.no_grad():
        output = model2(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        TEST["probabilities_2"].append(probabilities)
        predicted_class = probabilities.argmax().item()
    return int(predicted_class)

In [18]:
def predict3(image_path, model, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0) 
    image = image.to(DEVICE)
    with torch.no_grad():
        output = model2(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        TEST["probabilities_3"].append(probabilities)
        predicted_class = probabilities.argmax().item()
    return int(predicted_class)

In [19]:

df = pd.read_csv(CSV_FILE)
count=0
for index, row in tqdm(df.iterrows(), total=len(df)):
    # if count<=100:
        image_name = row['name']
        image_path = os.path.join(TEST_IMG_DIR, image_name + '.jpg')
        predicted_class = predict1(image_path, model1, transform)
        df.at[index, 'label'] = predicted_class

  0%|          | 0/2139 [00:00<?, ?it/s]

100%|██████████| 2139/2139 [09:02<00:00,  3.94it/s]


In [20]:
df = pd.read_csv(CSV_FILE)
count=0
for index, row in tqdm(df.iterrows(), total=len(df)):
    # if count<=100:
        image_name = row['name']
        image_path = os.path.join(TEST_IMG_DIR, image_name + '.jpg')
        predicted_class = predict2(image_path, model2, transform)
        df.at[index, 'label'] = predicted_class
        # count=count+1

  0%|          | 0/2139 [00:00<?, ?it/s]

100%|██████████| 2139/2139 [08:58<00:00,  3.97it/s]


In [44]:
print((TEST["probabilities_1"][0]))

tensor([0.3247, 0.1908, 0.4229, 0.0616])


In [21]:
df = pd.read_csv(CSV_FILE)
count=0
for index, row in tqdm(df.iterrows(), total=len(df)):
    # if count<=100:
        image_name = row['name']
        image_path = os.path.join(TEST_IMG_DIR, image_name + '.jpg')
        predicted_class = predict3(image_path, model3, transform)
        df.at[index, 'label'] = predicted_class

100%|██████████| 2139/2139 [06:34<00:00,  5.43it/s]


In [61]:
print(TEST["probabilities_1"][0])
print(TEST["probabilities_2"][0])
print(TEST["probabilities_3"][0])
print((TEST["probabilities_1"][0]/3+TEST["probabilities_2"][0]/3+TEST["probabilities_3"][0]/3))


tensor([0.3247, 0.1908, 0.4229, 0.0616])
tensor([2.6438e-08, 1.2990e-05, 9.9938e-01, 6.0353e-04])
tensor([2.6438e-08, 1.2990e-05, 9.9938e-01, 6.0353e-04])
tensor([0.1082, 0.0636, 0.8072, 0.0209])


In [53]:
print(len(TEST["probabilities_1"]))
print(len(TEST["probabilities_2"]))
print(len(TEST["probabilities_3"]))

2139
2139
2139


In [56]:
print(TEST["probabilities_1"][2138])

tensor([4.5091e-04, 7.2199e-02, 7.6974e-01, 1.5761e-01])


In [65]:
import csv
with open('sum.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["prediction"])
    for i in range(2138):
        sum=TEST["probabilities_1"][i]+TEST["probabilities_2"][i]+TEST["probabilities_3"][i]
        avg=sum/3
        writer.writerow([avg.argmax().item()])